In [1]:
! pip install optbinning

In [2]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV 
from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [5]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [6]:
# read data
train = pd.read_csv('/kaggle/input/dacon001/open/train.csv')
test = pd.read_csv('/kaggle/input/dacon001/open/test.csv')
sample_submission = pd.read_csv('/kaggle/input/dacon001/open/sample_submission.csv')
original_columns = test.columns.tolist()

In [7]:
# test데이터셋에 있는 변수만 선택 
train = train[test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY','AL']].copy()

> 모델학습

In [8]:
# """
# baseline model 

# >> seed_num: 176
# >> drop_features: 195
# Fold  1 RMSE : 1.054817
# Fold  2 RMSE : 1.055037
# Fold  3 RMSE : 0.994157
# Fold  4 RMSE : 1.082094
# Fold  5 RMSE : 1.078177
# Full RMSE score 1.201738
# feature
# ANONYMOUS_1            1144
# ZN                     1020
# ANONYMOUS_2             907
# YEAR_COMPONENT          894
# MO                      605
# FE                      564
# CR                      549
# PQINDEX                 492
# YEAR                    442
# TI                      355
# MN                      277
# H2O                     223
# AG                      152
# V                       145
# COMPONENT_ARBITRARY      80
# CO                       40
# Name: importance, dtype: int32
# """

# drop_features_vc1 = ['ZN_minus_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_CU', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'FE_divide_FE_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_max', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'FE_minus_FE_YEAR_COMPONENT_median', 'ZN_divide_ZN_YEAR_min', 'PQINDEX_divide_PQINDEX_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_sum', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_2_YEAR_max', 'FE_divide_FE_FE_woe_bin_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'V40_YEAR_median', 'V40_minus_V40_YEAR_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'NI_V40', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_max', 'ZN_minus_ZN_YEAR_min', 'FE_divide_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'CU_minus_CU_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_sum', 'ZN_divide_ZN_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_max', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'CU_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ZN_YEAR_COMPONENT_sum', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_median', 'CU_divide_CU_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'FE_divide_FE_FE_woe_bin_max', 'cluster_no_by_ANONYMOUS_1_median', 'V40_divide_V40_FE_woe_bin_sum', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_median', 'V40_divide_V40_YEAR_COMPONENT_max', 'V40_minus_V40_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_median', 'FE_minus_FE_FE_woe_bin_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ZN_minus_ZN_FE_woe_bin_sum', 'CU_minus_CU_FE_woe_bin_sum', 'V40', 'ANONYMOUS_1_ZN_woe_bin', 'FE_divide_FE_COMPONENT_ARBITRARY_min', 'CU_divide_CU_FE_woe_bin_sum', 'cluster_no_by_ANONYMOUS_2_mean', 'CU', 'V40_minus_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'V40_divide_V40_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_sum', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_V40_mean', 'FE_minus_FE_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_YEAR_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_FE_woe_bin_sum', 'FE_divide_FE_YEAR_sum', 'V40_minus_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_sum', 'ANONYMOUS_1_V40', 'ZN_divide_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_sum', 'ZN_divide_ZN_YEAR_median', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'V40_minus_V40_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CR_FE', 'FE_V40', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'PQINDEX_minus_PQINDEX_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_MO', 'CU_FE', 'ZN_minus_ZN_YEAR_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'V40_divide_V40_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'PQINDEX_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_max', 'CU_ZN', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_minus_V40_FE_woe_bin_min', 'NI_ZN', 'V40_divide_V40_FE_woe_bin_median', 'ZN_minus_ZN_YEAR_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'ANONYMOUS_1_ZN', 'PQINDEX_YEAR_median', 'NI', 'FE_ZN', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'CR_V40', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'MO_V40', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'cluster_no_by_ANONYMOUS_1_V40_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_median', 'ANONYMOUS_2_MO', 'PQINDEX_log1', 'CU_minus_CU_YEAR_COMPONENT_max', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_max', 'CU_divide_CU_FE_woe_bin_median']
# drop_features_vc2 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_CR', 'CU_minus_CU_YEAR_COMPONENT_max', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'FE', 'V40_minus_V40_YEAR_min', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max', 'CU_divide_CU_YEAR_COMPONENT_max', 'NI_ZN', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_max', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_COMPONENT_sum', 'FE_minus_FE_FE_woe_bin_median', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_sum', 'CU_divide_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_median', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'V40_divide_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_sum', 'CR_ZN', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'CR_V40', 'V40_minus_V40_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_sum', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_max', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'V40_divide_V40_YEAR_max', 'ANONYMOUS_2_MO', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'V40_divide_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'MO_NI', 'MO_ZN', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_median', 'CR_MO', 'CR_FE', 'ZN_divide_ZN_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'FE_divide_FE_YEAR_min', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'V40_woe_bin', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_median', 'CU_YEAR_COMPONENT_max', 'ANONYMOUS_2_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_sum', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_sum', 'NI', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'V40_YEAR_median', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'V40_divide_V40_FE_woe_bin_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_min', 'ANONYMOUS_1_CU', 'CU_divide_CU_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'V40', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_log1', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_sum']
# drop_features_vc3 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max']
# drop_features_vc4 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max']

# params =  {
#     'learning_rate': 0.07398,
#     'max_depth': 4.309,
#     'colsample_bytree': 0.4028,
#     'subsample': 0.4278,
#     'min_child_samples': 25.65,
#     'min_child_weight': 0.6138,
#     'min_split_gain': 0.7354,
#     'num_leaves': 62.68,
#     'reg_alpha': 0.2889,
#     'reg_lambda': 7.875
# }

# train_df,test_df = train_model_lgb_regressor(train,test,params,True,5,drop_features_vc1,seed_num=176)

In [9]:
def sub_model(train,test,params,stratified,num_folds,drop_features,seed_num):
    
    # start log 
    # print('-'*50)
    # print('>> seed_num:',seed_num)   
    # print('>> drop_features:',len(drop_features))
    
    seed_everything(1)
    
    # Divide in training/validation and test data
    train_df = train.copy()
    test_df = test.copy()
    
    train_df['AL'] = np.log(train_df['AL']+1)

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        test_df[each] = encoder.fit_transform(test_df[each])

    # set training options
    stratified = stratified
    num_folds = num_folds

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1)

    # Create arrays and dataframes to store results
    oof_preds_lgb = np.zeros(train_df.shape[0])
    sub_preds_lgb = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY','AL']+drop_features]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['YEAR'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['AL'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['AL'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMRegressor(
            
            learning_rate = params['learning_rate'],
            num_leaves = int(round(params['num_leaves'])),
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            max_depth = int(round(params['max_depth'])),
            reg_alpha = params['reg_alpha'],
            reg_lambda = params['reg_lambda'],
            min_split_gain = params['min_split_gain'],
            min_child_weight = params['min_child_weight'],
            min_child_samples = int(round(params['min_child_samples'])),    
            
            n_jobs = -1,
            n_estimators = 10000,            
            random_state = seed_num,
            silent=-1,
            deterministic=True,
            verbose=-1
        )
        
        with warnings.catch_warnings():
            
            warnings.filterwarnings('ignore')

            clf.fit(
                  train_x
                , train_y
                , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                , eval_metric= 'rmse'
                , verbose= -1
                , early_stopping_rounds= 500
            )

        oof_preds_lgb[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)
        sub_preds_lgb += clf.predict(test_df[feats], num_iteration=clf.best_iteration_) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        # print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds_lgb[valid_idx])))
        # print('Fold %2d RMSE : %.6f' % (n_fold + 1, mean_squared_error(valid_y, oof_preds_lgb[valid_idx])**0.5))               

    err = mean_squared_error(train_df['Y_LABEL'], oof_preds_lgb)**0.5  
    feature_importance_df = feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False)
    # display(feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False).head(30))
    
    return err, feature_importance_df

In [10]:
def train_model_lgb_regressor(train,test,params,stratified,num_folds,drop_features,seed_num):
    
    # start log 
    print('-'*50)
    print('>> seed_num:',seed_num)   
    print('>> drop_features:',len(drop_features))
    
    new_feature = train.columns[train.columns.str.contains('new')].tolist()
    
    seed_everything(1)
    
    # Divide in training/validation and test data
    train_df = train.copy()
    test_df = test.copy()
    
    train_df['AL'] = np.log(train_df['AL']+1)

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        test_df[each] = encoder.fit_transform(test_df[each])

    # set training options
    stratified = stratified
    num_folds = num_folds

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1)

    # Create arrays and dataframes to store results
    oof_preds_lgb = np.zeros(train_df.shape[0])
    sub_preds_lgb = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY','AL']+drop_features+new_feature]
    print('>> # of features:',len(feats))

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['YEAR'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['AL'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['AL'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMRegressor(
            
            learning_rate = params['learning_rate'],
            num_leaves = int(round(params['num_leaves'])),
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            max_depth = int(round(params['max_depth'])),
            reg_alpha = params['reg_alpha'],
            reg_lambda = params['reg_lambda'],
            min_split_gain = params['min_split_gain'],
            min_child_weight = params['min_child_weight'],
            min_child_samples = int(round(params['min_child_samples'])),    
            
            n_jobs = -1,
            n_estimators = 10000,            
            random_state = seed_num,
            silent=-1,
            deterministic=True,
            verbose=-1
        )
        
        with warnings.catch_warnings():
            
            warnings.filterwarnings('ignore')

            clf.fit(
                  train_x
                , train_y
                , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                , eval_metric= 'rmse'
                , verbose= -1
                , early_stopping_rounds= 500
            )

        oof_preds_lgb[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)
        sub_preds_lgb += clf.predict(test_df[feats], num_iteration=clf.best_iteration_) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        # print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds_lgb[valid_idx])))
        # print('Fold %2d RMSE : %.6f' % (n_fold + 1, mean_squared_error(valid_y, oof_preds_lgb[valid_idx])**0.5))               

    global_err = mean_squared_error(train_df['Y_LABEL'], oof_preds_lgb)**0.5
    
    feature_importance_df = feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False)
    print('RMSE(asis) %.6f' % global_err)

    # Write submission file and plot feature importance
    # train_df['Y_LABEL_lgb'] = oof_preds_lgb
    # test_df['Y_LABEL_lgb'] = sub_preds_lgb   

    rst_list = []
    vi_list = []
    for i in range(5):
        err, vi = sub_model(train,test,params,True,5,drop_features,seed_num=i)
        rst_list.append({'seed':i,'err':err})    
        vi_list.append(vi)
        
    print('>> new try')
    print('# of new',len(new_feature))
    print('RMSE(mean):',pd.DataFrame(rst_list)['err'].mean()) 
    print('RMSE(best):',pd.DataFrame(rst_list)['err'].min())
    print('RMSE(stdv):',pd.DataFrame(rst_list)['err'].std())   
    display(pd.DataFrame(vi_list).T.apply(lambda x: x.mean(),axis=1).reset_index(name='vi').head(20))
    
    return pd.DataFrame(rst_list),pd.DataFrame(vi_list)

In [11]:
train['YEAR_COMPONENT'] = train[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
test['YEAR_COMPONENT'] = test[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)

In [18]:
"""
baseline model 

>> seed_num: 176
>> drop_features: 195
Fold  1 RMSE : 1.054495
Fold  2 RMSE : 1.053187
Fold  3 RMSE : 0.998778
Fold  4 RMSE : 1.083832
Fold  5 RMSE : 1.082376
Full RMSE score 1.201391
"""

drop_features_vc1 = ['ZN_minus_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_CU', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'FE_divide_FE_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_max', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'FE_minus_FE_YEAR_COMPONENT_median', 'ZN_divide_ZN_YEAR_min', 'PQINDEX_divide_PQINDEX_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_sum', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_2_YEAR_max', 'FE_divide_FE_FE_woe_bin_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'V40_YEAR_median', 'V40_minus_V40_YEAR_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'NI_V40', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_max', 'ZN_minus_ZN_YEAR_min', 'FE_divide_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'CU_minus_CU_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_sum', 'ZN_divide_ZN_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_max', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'CU_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ZN_YEAR_COMPONENT_sum', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_median', 'CU_divide_CU_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'FE_divide_FE_FE_woe_bin_max', 'cluster_no_by_ANONYMOUS_1_median', 'V40_divide_V40_FE_woe_bin_sum', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_median', 'V40_divide_V40_YEAR_COMPONENT_max', 'V40_minus_V40_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_median', 'FE_minus_FE_FE_woe_bin_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ZN_minus_ZN_FE_woe_bin_sum', 'CU_minus_CU_FE_woe_bin_sum', 'V40', 'ANONYMOUS_1_ZN_woe_bin', 'FE_divide_FE_COMPONENT_ARBITRARY_min', 'CU_divide_CU_FE_woe_bin_sum', 'cluster_no_by_ANONYMOUS_2_mean', 'CU', 'V40_minus_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'V40_divide_V40_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_sum', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_V40_mean', 'FE_minus_FE_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_YEAR_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_FE_woe_bin_sum', 'FE_divide_FE_YEAR_sum', 'V40_minus_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_sum', 'ANONYMOUS_1_V40', 'ZN_divide_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_sum', 'ZN_divide_ZN_YEAR_median', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'V40_minus_V40_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CR_FE', 'FE_V40', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'PQINDEX_minus_PQINDEX_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_MO', 'CU_FE', 'ZN_minus_ZN_YEAR_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'V40_divide_V40_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'PQINDEX_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_max', 'CU_ZN', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_minus_V40_FE_woe_bin_min', 'NI_ZN', 'V40_divide_V40_FE_woe_bin_median', 'ZN_minus_ZN_YEAR_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'ANONYMOUS_1_ZN', 'PQINDEX_YEAR_median', 'NI', 'FE_ZN', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'CR_V40', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'MO_V40', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'cluster_no_by_ANONYMOUS_1_V40_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_median', 'ANONYMOUS_2_MO', 'PQINDEX_log1', 'CU_minus_CU_YEAR_COMPONENT_max', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_max', 'CU_divide_CU_FE_woe_bin_median']
drop_features_vc2 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_CR', 'CU_minus_CU_YEAR_COMPONENT_max', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'FE', 'V40_minus_V40_YEAR_min', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max', 'CU_divide_CU_YEAR_COMPONENT_max', 'NI_ZN', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_max', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_COMPONENT_sum', 'FE_minus_FE_FE_woe_bin_median', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_sum', 'CU_divide_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_median', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'V40_divide_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_sum', 'CR_ZN', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'CR_V40', 'V40_minus_V40_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_sum', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_max', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'V40_divide_V40_YEAR_max', 'ANONYMOUS_2_MO', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'V40_divide_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'MO_NI', 'MO_ZN', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_median', 'CR_MO', 'CR_FE', 'ZN_divide_ZN_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'FE_divide_FE_YEAR_min', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'V40_woe_bin', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_median', 'CU_YEAR_COMPONENT_max', 'ANONYMOUS_2_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_sum', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_sum', 'NI', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'V40_YEAR_median', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'V40_divide_V40_FE_woe_bin_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_min', 'ANONYMOUS_1_CU', 'CU_divide_CU_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'V40', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_log1', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_sum']
drop_features_vc3 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max']
drop_features_vc4 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max']

params =  {
    'learning_rate': 0.07398,
    'max_depth': 4.309,
    'colsample_bytree': 0.4028,
    'subsample': 0.4278,
    'min_child_samples': 25.65,
    'min_child_weight': 0.6138,
    'min_split_gain': 0.7354,
    'num_leaves': 62.68,
    'reg_alpha': 0.2889,
    'reg_lambda': 7.875
}

train2 = train.copy()
test2 = test.copy()

train2['new1'] = train2['ANONYMOUS_1'] / train2['ANONYMOUS_2'] 
test2['new1'] = test2['ANONYMOUS_1'] / test2['ANONYMOUS_2'] 
# train2['new2'] = train2['FE'] / train2['ZN'] 
# test2['new2'] = test2['FE'] / test2['ZN']
# train2['new3'] = train2['FE'] - train2['ZN'] 
# test2['new3'] = test2['FE'] - test2['ZN']
# train2['new4'] = train2['FE'] + train2['ZN'] 
# test2['new4'] = test2['FE'] + test2['ZN']

drop_features = []

a1,vi = train_model_lgb_regressor(train2,test2,params,True,5,drop_features,176)

--------------------------------------------------
>> seed_num: 176
>> drop_features: 0
>> # of features: 19
RMSE(asis) 1.203415
>> new try
# of new 1
RMSE(mean): 1.2035071820083432
RMSE(best): 1.2025294157563677
RMSE(stdv): 0.000901160329973325


,feature,vi
0,new1,751.6
1,V40,774.8
2,ANONYMOUS_1,864.4
3,FE,703.2
4,ZN,659.8
5,YEAR_COMPONENT,679.0
6,CU,535.2
7,ANONYMOUS_2,508.4
8,PQINDEX,563.2
9,YEAR,382.4


In [13]:
"""
--------------------------------------------------
>> seed_num: 176
>> drop_features: 195
ncol 16
RMSE(as-is) 1.201738
>> new try
# of new 1
RMSE(to-be): 1.2031642053847027
RMSE(best): 1.2015763975726006
RMSE(std): 0.0008509228737894823
add Codeadd Markdown
"""

'\n--------------------------------------------------\n>> seed_num: 176\n>> drop_features: 195\nncol 16\nRMSE(as-is) 1.201738\n>> new try\n# of new 1\nRMSE(to-be): 1.2031642053847027\nRMSE(best): 1.2015763975726006\nRMSE(std): 0.0008509228737894823\nadd Codeadd Markdown\n'

In [14]:
# train_df['AL'] = np.exp(train_df['AL'])-1
# train_df['Y_LABEL_lgb'] = np.exp(train_df['Y_LABEL_lgb'])-1

# train_df['pred'] = train_df['Y_LABEL_lgb']
# train_df['err'] = train_df['AL'] - train_df['pred']

# a1 = pd.read_csv('/kaggle/input/dacon001/open/train.csv')
# a1 = a1.drop(columns=['Y_LABEL','AL'])
# rst = train_df[['ID','YEAR','Y_LABEL_lgb','pred','err','Y_LABEL','AL']].merge(a1,on=['ID','YEAR'])
# rst.to_excel('rst5.xlsx')
# rst.head()